In [1]:
from nixtlats import TimeGPT

timegpt = TimeGPT(
    # defaults to os.environ.get("TIMEGPT_TOKEN")
    token = 'JxQF7I2vM8Hpbue718L2gGCfG1gflGqWJCkWb6gRjVL2r9p0TPhDmoOIMAAOgGdQG1tFUDFKgkq52cbyAM1H8Li1jTg9KtV6zWKGuejojqMV6i5wzTgppL8KnsvfcfpYidXbBRgqSLlVilfjjvWwTIMFixvHbIu1cEBdP8CUcjeRlsWlg2ZkYlLr3iiG2RojTAG4BKHRnSX2S7uIpaKMjBZcwRlVpf8KoUFXfAaQj2a5UW98o5IMPe196Iz2uqwD'
)


Nixtla's SDK for TimeGPT takes pandas DF as inputs. Here we are using an Electricity Price Forecasting (EPF) dataset.

The input data will require the following columns:

series identifier (column name specified with #id_col argument)
datestamps (column name specified with #time_col argument)
target historical values (column name specified with #target_col argument)

In [2]:
timegpt.validate_token()


INFO:nixtlats.timegpt:Happy Forecasting! :), If you have questions or need support, please email ops@nixtla.io


True

In [137]:
from utils import data_handling, helpers

window_size = 96
pred_length = pred_length = (96)#, 192, 336, 720)

data_dict = data_handling.load_electricity()
_, _, dataloader_test = data_handling.convert_data(data_dict, window_size, pred_length)


Feature batch shape: torch.Size([32, 96, 348])


In [15]:
input, target = next(iter(dataloader_test))
input[0,:].shape
#target[0][0,:]

torch.Size([96, 348])

In [119]:
len(dataloader_test)

67

In [12]:
import torch
import pandas as pd
import numpy as np

# Simulate a PyTorch tensor with random data
tensor = torch.rand(96, 348)

# Convert the tensor to a numpy array and then flatten it
targets = input[0,:].numpy().flatten()

# Create a list of IDs repeated for each time step
ids = np.tile(np.arange(1, 349), 96)  # IDs from 1 to 348

time_steps = np.repeat(timestamps, 348)


# Create a list of time steps for each ID
#time_steps = np.repeat(np.arange(96), 348)

# Create a DataFrame
df = pd.DataFrame({
    'Time': time_steps,
    'ID': ids,
    'Target': targets
})

# The DataFrame has one 'Time' column, one 'ID' column, and one 'Target' column
small_df = df[df["ID"] < 5]
small_df



small_df["Time"] = small_df["Time"].astype(str).copy()

small_df

/tmp/ipykernel_47855/3006045963.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_df["Time"] = small_df["Time"].astype(str).copy()


,Time,ID,Target
0,2023-04-01 00:00:00,1,3.719513
1,2023-04-01 00:00:00,2,1.325190
2,2023-04-01 00:00:00,3,0.071790
3,2023-04-01 00:00:00,4,0.141977
348,2023-04-01 01:00:00,1,3.807355
...,...,...,...
32715,2023-04-04 22:00:00,4,0.667256
33060,2023-04-04 23:00:00,1,3.192456
33061,2023-04-04 23:00:00,2,1.131707
33062,2023-04-04 23:00:00,3,0.071790


In [120]:
67*32

2144

In [145]:
# len(dataloader) = 67 and num_batches = 32 -> this has to be done 2100 times

def reshape_dataloader_to_gpt_input(dataloader, num_ids=None):

    input, target = next(iter(dataloader))

    input =   input[0][:,0:num_ids]
    target = target[0][:,0:num_ids]

    horizon = input.size(0)
    if num_ids == None:
        num_ids = input.size(1)


    timestamps = pd.date_range(start='2023-04-01', periods=horizon, freq='H')
    time_steps = np.repeat(timestamps, num_ids)

    ids = np.tile(np.arange(0, num_ids), horizon) 


    input = input.numpy().flatten()


    df = pd.DataFrame({
    'Time': time_steps,
    'ID': ids,
    'Target': input
    })
    return df, target


def reshape_gpt_output_to_eval_tensor(timegpt_fcst_df):
    temp = timegpt_fcst_df

    num_ids = temp["ID"].nunique()
    horizon = temp["Time"].nunique()

    preds = temp["TimeGPT"]
    preds_tensor = torch.tensor(preds.to_list())
    preds_tensor_2d = preds_tensor.view(horizon, num_ids)
    

    return preds_tensor_2d


df, target = reshape_dataloader_to_gpt_input(dataloader_test, 10)


timegpt_fcst_df = timegpt.forecast(df=df, h=96, freq="H", finetune_steps=0, id_col='ID', time_col='Time', target_col='Target')


preds = reshape_gpt_output_to_eval_tensor(timegpt_fcst_df)


error = helpers.mean_squared_error(preds, target)

error

INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Calling Forecast Endpoint...


tensor(3.5620)

In [97]:


#fcst_df = timegpt.forecast(df=df, h=36, model='timegpt-1-long-horizon', time_col='timestamp', target_col='value')


INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Calling Forecast Endpoint...


In [117]:
timegpt_fcst_df

,ID,Time,TimeGPT
0,1,2023-04-05 00:00:00,3.317608
1,1,2023-04-05 01:00:00,3.541721
2,1,2023-04-05 02:00:00,3.395764
3,1,2023-04-05 03:00:00,3.447697
4,1,2023-04-05 04:00:00,3.436582
...,...,...,...
379,4,2023-04-08 19:00:00,0.559763
380,4,2023-04-08 20:00:00,0.976372
381,4,2023-04-08 21:00:00,0.972878
382,4,2023-04-08 22:00:00,0.691888


In [33]:
preds = timegpt_fcst_df["TimeGPT"]
outputs = preds.tolist()
len(outputs)


384

In [36]:
small_df

,Time,ID,Target
0,2023-04-01 00:00:00,1,3.719513
1,2023-04-01 00:00:00,2,1.325190
2,2023-04-01 00:00:00,3,0.071790
3,2023-04-01 00:00:00,4,0.141977
348,2023-04-01 01:00:00,1,3.807355
...,...,...,...
32715,2023-04-04 22:00:00,4,0.667256
33060,2023-04-04 23:00:00,1,3.192456
33061,2023-04-04 23:00:00,2,1.131707
33062,2023-04-04 23:00:00,3,0.071790


In [63]:
x = timegpt_fcst_df["ID"][-96:] = 4
x

/tmp/ipykernel_47855/2288874174.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x = timegpt_fcst_df["ID"][-96:] = 4


4

In [93]:
preds = timegpt_fcst_df["TimeGPT"]
preds_list = torch.tensor(preds.to_list())
preds_list = preds_list.view(96,4)
preds_list.shape

torch.Size([96, 4])

In [94]:
small_targets = target[0][0,:,0:4]
small_targets.shape

torch.Size([96, 4])

In [96]:
helpers.mean_squared_error(preds_list, small_targets) / (96*4)

tensor(0.0137)